<a href="https://colab.research.google.com/github/tobenson21/Data_Enginner/blob/main/tareaPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraccion y transformacion de dataset de github para realizar inyeccion de la data a bigquery y luego generar visualizaciones con looker studio:

In [ ]:
#Traer dataset de github a colab
!wget https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
!wget https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
!wget https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv

--2024-12-14 18:56:30--  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819904 (1.7M) [text/plain]
Saving to: ‘time_series_covid19_confirmed_global.csv’

time_series_covid19 100%[===================>]   1.74M  --.-KB/s    in 0.04s   

2024-12-14 18:56:31 (40.6 MB/s) - ‘time_series_covid19_confirmed_global.csv’ saved [1819904/1819904]

--2024-12-14 18:56:31--  https://raw.githubusercontent.com/CSSEGISandData/COVID-19/refs/heads/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133,

In [ ]:
#Importar la libreria de Pandas
import pandas as pd

In [ ]:
#Leer los dataset
confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv',sep= ',')
deaths = pd.read_csv('time_series_covid19_deaths_global.csv',sep= ',')
recovered = pd.read_csv('time_series_covid19_recovered_global.csv',sep= ',')

Ver cantidad de columna y filas de los 3 dataset

In [ ]:
confirmed.shape

(289, 1147)

In [ ]:
deaths.shape

(274, 1147)

In [ ]:
recovered.shape

(274, 1147)

In [ ]:
#Eliminar columna Province/State de los dataset
del confirmed['Province/State']
del deaths['Province/State']
del recovered['Province/State']

In [ ]:
#Realizacion de pivot de las columnas fechas
confirmed = pd.melt(confirmed, id_vars=['Country/Region', 'Lat', 'Long'], var_name='date', value_name='qty_confirmed')
deaths = pd.melt(deaths, id_vars=['Country/Region', 'Lat', 'Long'], var_name='date', value_name='qty_deaths')
recovered = pd.melt(recovered, id_vars=['Country/Region', 'Lat', 'Long'], var_name='date', value_name='qty_recovered')

In [ ]:
confirmed.head()

,Country/Region,Lat,Long,date,qty_confirmed
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


In [ ]:
#Unificar datasets
df_covid_unified = pd.concat([confirmed, deaths, recovered], ignore_index=True)
df_covid_unified

,Country/Region,Lat,Long,date,qty_confirmed,qty_deaths,qty_recovered
0,Afghanistan,33.939110,67.709953,1/22/20,0.0,NaN,NaN
1,Albania,41.153300,20.168300,1/22/20,0.0,NaN,NaN
2,Algeria,28.033900,1.659600,1/22/20,0.0,NaN,NaN
3,Andorra,42.506300,1.521800,1/22/20,0.0,NaN,NaN
4,Angola,-11.202700,17.873900,1/22/20,0.0,NaN,NaN
...,...,...,...,...,...,...,...
973831,West Bank and Gaza,31.952200,35.233200,3/9/23,NaN,NaN,0.0
973832,Winter Olympics 2022,39.904200,116.407400,3/9/23,NaN,NaN,0.0
973833,Yemen,15.552727,48.516388,3/9/23,NaN,NaN,0.0
973834,Zambia,-13.133897,27.849332,3/9/23,NaN,NaN,0.0


In [ ]:
# Cambiar nombre de columna, en bigquey no se permite el signo '/'
df_covid_unified = df_covid_unified.rename(columns = {'Country/Region': 'country_region'})

In [ ]:
#Formatear fecha, en BigQuery sólo es capaz de entender formatos de fecha separadas por "-"
df_covid_unified['date'] = pd.to_datetime(df_covid_unified['date'], format='%m/%d/%y')

In [ ]:
df_covid_unified.head()

,country_region,Lat,Long,date,qty_confirmed,qty_deaths,qty_recovered
0,Afghanistan,33.93911,67.709953,2020-01-22,0.0,NaN,NaN
1,Albania,41.15330,20.168300,2020-01-22,0.0,NaN,NaN
2,Algeria,28.03390,1.659600,2020-01-22,0.0,NaN,NaN
3,Andorra,42.50630,1.521800,2020-01-22,0.0,NaN,NaN
4,Angola,-11.20270,17.873900,2020-01-22,0.0,NaN,NaN


In [ ]:
#Ordenar dataset por fecha
df_covid_unified.sort_values(by='date', inplace=True)
df_covid_unified

,country_region,Lat,Long,date,qty_confirmed,qty_deaths,qty_recovered
0,Afghanistan,33.939110,67.709953,2020-01-22,0.0,NaN,NaN
330436,Dominican Republic,18.735700,-70.162700,2020-01-22,NaN,0.0,NaN
330435,Dominica,15.415000,-61.371000,2020-01-22,NaN,0.0,NaN
330434,Djibouti,11.825100,42.590300,2020-01-22,NaN,0.0,NaN
330433,Diamond Princess,0.000000,0.000000,2020-01-22,NaN,0.0,NaN
...,...,...,...,...,...,...,...
660459,Comoros,-11.645500,43.333300,2023-03-09,NaN,161.0,NaN
660458,Colombia,4.570900,-74.297300,2023-03-09,NaN,142339.0,NaN
660457,China,29.183200,120.093400,2023-03-09,NaN,1.0,NaN
660509,Holy See,41.902900,12.453400,2023-03-09,NaN,0.0,NaN


In [ ]:
#Remplazar datos nulos por 0
df_covid_unified.fillna(0,inplace= True)
df_covid_unified

,country_region,Lat,Long,date,qty_confirmed,qty_deaths,qty_recovered
0,Afghanistan,33.939110,67.709953,2020-01-22,0.0,0.0,0.0
330436,Dominican Republic,18.735700,-70.162700,2020-01-22,0.0,0.0,0.0
330435,Dominica,15.415000,-61.371000,2020-01-22,0.0,0.0,0.0
330434,Djibouti,11.825100,42.590300,2020-01-22,0.0,0.0,0.0
330433,Diamond Princess,0.000000,0.000000,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...,...
660459,Comoros,-11.645500,43.333300,2023-03-09,0.0,161.0,0.0
660458,Colombia,4.570900,-74.297300,2023-03-09,0.0,142339.0,0.0
660457,China,29.183200,120.093400,2023-03-09,0.0,1.0,0.0
660509,Holy See,41.902900,12.453400,2023-03-09,0.0,0.0,0.0


In [ ]:
#Instalar Bigquery
!pip install --upgrade google-cloud-bigquery

In [ ]:
#Usar libreria de autenticacion
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Autentificado')

Autentificado


In [ ]:
# Establecer el proyecto
project_id = 'datapipelinework'
client = bigquery.Client(project = project_id)

In [ ]:
# Definir el nombre de la tabla en BigQuery
dataset_id = 'datapipelinework.df_covid_unified'
table_id = 'pipelines_home_work'

# Subir el DataFrame a BigQuery
df_covid_unified.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

<ipython-input-54-21682b99096e>:6: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_covid_unified.to_gbq(destination_table=f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 7384.34it/s]
